In [0]:
-- Moving this one to pipe syntax made it 15% faster (130ms  over 880ms)

from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_06_input.txt', format => 'text', wholeText => true) as t (value)
/*from values ('123 328  51 64 ' || '\n'
           ||  ' 45 64  387 23 ' || '\n'
           ||  '  6 98  215 314' || '\n'
           ||  '*   +   *   +  ') as t (value) */
|> select split(value, '\n') as lines
        , regexp_extract_all(element_at(lines, -1), r'(\s+)') as ope_line
|> select aggregate( transform( ope_line
                              , (t, i) -> struct( filter(split(element_at(lines, -1), r'\s+'), f -> f <> '')[i] as s
                                                , length(t) + case i when size(ope_line) - 1 then 1 else 0 end as l))
                   , array()::array<struct<s:string,b:int,l:int,c:int>>
                   , (acc, x) -> array_append(acc, struct( x.s
                                                         , coalesce(try_element_at(acc.c, -1), 0) + 1 as b
                                                         , x.l
                                                         , coalesce(try_element_at(acc.c, -1), 0) + 1 + x.l as c) )
                   , acc -> transform( acc
                                     , t -> struct( t.s
                                                  , transform( sequence(0, array_size(lines) - 2)
                                                             , u -> substring(lines[u] from t.b for t.l)) as n
                                                  ) )
                   ) as blocks
|> select explode(blocks) as block
|> aggregate sum(aggregate( transform(block.n, t -> t::bigint)
                          , (block.s = '*')::bigint
                          , (agg, y) -> case block.s when '*' then agg * y else agg + y end)) as part1
           , sum(aggregate( transform( sequence(0, length(block.n[0]) - 1)
                                     , i -> array_join(transform( sequence(0, array_size(block.n) - 1)
                                                                , j -> split(block.n[j], '')[i]), '')::bigint)
                          , (block.s = '*')::bigint
                          , (agg, y) -> case block.s when '*' then agg * y else agg + y end)) as part2;